In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))
print(text[:30])

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

In [ ]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for ch, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('hi: there'))
print(decode(encode('hi: there')))

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

In [ ]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F 
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        #idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx)  #(B, T, C) batch, block, channel or vocab

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            # print("logits shape", logits.shape)
            # print("target shape", targets.shape)
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # print("after logits shape", logits.shape)
            # print("after target shape", targets.shape)

            loss = F.cross_entropy(logits, targets)
        return logits, loss
        
    def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
        for i in range(max_new_tokens):
            # get the predicion
            logits, loss = self(idx)  #[1, 1, 65]
            # focus only on the last time step
            logits1 = logits[:, -1, :] #becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits1, dim=-1)  #(B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat ((idx, idx_next), dim = 1)  #(B, T+1)
            # print("logits", logits.shape)
            # print("logits1", logits1)
            # print("probs", probs)
            # print("idx_next", idx_next)
        return idx

    
    

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))




In [ ]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
lossa = []
for steps in range(3000):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    lossa.append(loss)

plt.plot(torch.tensor(lossa).detach().numpy())

In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))

In [ ]:
torch.manual_seed(1337)
B, T, C = 4, 8 ,2
x = torch.randn(B, T, C)  # batch time channels
x.shape

In [ ]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        r = x[b,:t]
        # print("this", r)
        xprev = x[b,:t+1]
        # print("t+1", xprev)
        xbow[b,t] = torch.mean(xprev, 0)
        
xbow.shape

In [ ]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x #(T, T) ~>(B, T, T) @ (B, T, C) ===> (B, T, C)

torch.allclose(xbow, xbow2)


In [ ]:
# version 3
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

In [ ]:
# version 4

torch.manual_seed(1337)
B,T,C=4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  #(B, T, 16)
q = query(x)  #(B, T, 16)
wei = q @ k.transpose(-2, -1)  #(B, T, 16 @ (B, 16, T) ==> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
# out = wei @ x

out.shape

### 📊 Understanding `dim` in `torch.softmax()` for a 3D Tensor (`[batch, rows, features]`)

| **`dim`**       | **Applies softmax across:**       | **Example Effect**                                      |
|-----------------|-----------------------------------|---------------------------------------------------------|
| `dim=0`         | Across batches                   | Compares values at the same `[row][feature]` across batches |
| `dim=1`         | Across rows in a batch           | Compares values at the same `[feature]` per row        |
| `dim=2` or `-1` | Across features per row          | Compares values within each row   

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print("a=")
print(a)
print('--')
print("b=")
print(b)
print('--')
print("c=")
print(c)



### Layer Norm

documentation->
https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html

In [ ]:
class LayerNorm:
    
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    
    def __call__(self, x):
        xmean = x.mean(1, keepdim=True)
        xvar = x.var(1, keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # noralize to unit variance
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self. beta]
    
torch.manual_seed(1337)
module = LayerNorm(100)
x = torch.randn(32, 100)
x = module(x)
x.shape

In [ ]:
x[:,0].mean(), x[:,0].std()

In [ ]:
x[0,:].mean(), x[0,:].std()